# Planeamento de horários com satisfação de restrições

Este caderno mostra, passo a passo, como modelar e resolver um problema simples de construção de horários usando satisfação de restrições. As decisões e justificações estão documentadas ao longo das secções para clarificar a estratégia seguida.

### Introdução
- **Contexto**: projeto aborda a elaboração de horários semanais para três turmas, conciliando docentes, salas e blocos temporais através da satisfação de restrições.
- **Propósito**: construir um modelo CSP reprodutível e analisar resultados que apresentem horários viáveis, justificando todas as decisões de modelação.

### Equipa
- Gonçalo Tierri, 23020
- Fábio Costa, 22997
- Lino Azevedo, 23015
- Fábio Fernandes, 22996
- Luís Freitas, 23008

### Desenho do agente
- **Formulação CSP**: cada aula (`UCxx_1`, `UCxx_2`) representa uma variável com valores possíveis `(bloco temporal, sala)`, garantindo duas sessões semanais por UC.
- **Domínios**: `domainForLesson` filtra blocos incompatíveis com disponibilidades dos docentes, impõe salas obrigatórias e direciona aulas online para a sala virtual `Online`.
- **Restrições duras**: `addHardConstrains` evita sobreposições por docente e por turma, impede partilha de sala física no mesmo bloco, limita cada turma a três blocos por dia e segue o mapa dia→blocos através de auxiliares como `noOverlappingBlock` e `threeClassesMax`.
- **Restrições suaves e heurísticas**: `evaluateSolution` controla preferências — blocos consecutivos por turma, presença em exactamente quatro dias, sessões de uma UC em dias distintos, penalização por uso de várias salas — com pesos (10–100) que orientam a minimização das penalizações.
- **Estratégia do solver**: execuções sucessivas de `MinConflictsSolver` com ordenação por tamanho de domínio (`orderedLessons`) e sementes determinísticas exploram o espaço de soluções; a melhor solução fica registada e o processo termina quando a penalização atinge zero.


## Visão geral do fluxo de trabalho

1. Preparar dependências e importar bibliotecas.
2. Definir os dados do problema e as restrições conhecidas.
3. Construir os domínios e funções auxiliares que suportam as restrições.
4. Especificar restrições fortes (obrigatórias) e restrições suaves (penalizações).
5. Executar o solver e inspecionar a melhor solução encontrada.


## Dependências

Execute a célula seguinte apenas se estiver a correr o notebook num ambiente novo. Usamos `python-constraint`, que oferece diferentes algoritmos de procura para problemas de satisfação de restrições.


In [ ]:
%pip install --quiet --upgrade pip
%pip install --quiet python-constraint


## Importações e configuração

Carregamos as bibliotecas necessárias. O solver `MinConflictsSolver` é adequado para problemas com muitas restrições suaves porque tenta minimizar conflitos iterativamente.


In [ ]:
import random

from constraint import MinConflictsSolver, Problem
from itertools import combinations


## Dados de base do problema

Estas listas definem a estrutura temporal (dias/blocos), as turmas, os docentes, as unidades curriculares (UC) e as salas disponíveis. São os dados fornecidos à partida pelo domínio do problema.


In [ ]:
days = 'Segunda Terça Quarta Quinta Sexta'.split()
classGroups = 't01 t02 t03'.split()
blocks = 'B01 B02 B03 B04 B05 B06 B07 B08 B09 B10 B11 B12 B13 B14 B15 B16 B17 B18 B19 B20'.split()
lecturers = 'jo mike rob sue'.split()
courses = 'UC11 UC12 UC13 UC14 UC15 UC21 UC22 UC23 UC24 UC25 UC31 UC32 UC33 UC34 UC35'.split()
rooms = 'A C G Lab01 Lab02'.split()


## Mapas entre turmas, docentes e salas

Agrupamos as UC por turma, listamos as UC atribuídas a cada docente e especificamos restrições adicionais: salas obrigatórias, indisponibilidades de docentes e o mapeamento de blocos por dia.


In [ ]:
courseClasses = {
    't01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15'],
    't02': ['UC21', 'UC22', 'UC23', 'UC24', 'UC25'],
    't03': ['UC31', 'UC32', 'UC33', 'UC34', 'UC35'],
}

classLecturers = {
    'jo': ['UC11', 'UC21', 'UC22', 'UC31'],
    'mike': ['UC12', 'UC23', 'UC32'],
    'rob': ['UC13', 'UC14', 'UC24', 'UC33'],
    'sue': ['UC15', 'UC25', 'UC34', 'UC35'],
}

classRooms = {
    'UC22': ['Lab01'],
    'UC14': ['Lab01'],
}

timeRestrictions = {
    'mike': ['B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20'],
    'rob': ['B01', 'B02', 'B03', 'B04'],
    'sue': ['B09', 'B10', 'B11', 'B12', 'B17', 'B18', 'B19', 'B20'],
}

daysBlocks = {
    'Segunda': ['B01', 'B02', 'B03', 'B04'],
    'Terça': ['B05', 'B06', 'B07', 'B08'],
    'Quarta': ['B09', 'B10', 'B11', 'B12'],
    'Quinta': ['B13', 'B14', 'B15', 'B16'],
    'Sexta': ['B17', 'B18', 'B19', 'B20'],
}

onlineLessons = {'UC21_2', 'UC31_2'}
physicalRooms = [room for room in rooms if room != 'Online']


## Construção dos domínios

Enumeramos todas as aulas a agendar e criamos mapas auxiliares (UC → docente, UC → turma, bloco → dia) para agilizar verificações posteriores.


In [ ]:
lessonsToSchedule = [f'{courseCode}_{index}' for courseCode in courses for index in (1, 2)]

courseToTeacher = {
    courseCode: lecturerName
    for lecturerName, courseList in classLecturers.items()
    for courseCode in courseList
}

courseToGroup = {
    courseCode: groupName
    for groupName, courseList in courseClasses.items()
    for courseCode in courseList
}

blockToDayMap = {
    block: day
    for day, blocksInDay in daysBlocks.items()
    for block in blocksInDay
}


### Domínio de cada aula

A função abaixo gera todas as combinações possíveis de (bloco, sala) para uma aula. Respeitamos indisponibilidades dos docentes, garantimos salas específicas quando necessário e distinguimos aulas online que não consomem salas físicas.


In [ ]:
def domainForLesson(lessonName):
    """Devolve o domínio de pares (bloco, sala) admissíveis para uma aula."""
    courseCode = lessonName.split('_')[0]
    teacherName = courseToTeacher[courseCode]

    teacherBlockedBlocks = set(timeRestrictions.get(teacherName, []))
    allowedBlocks = [block for block in blocks if block not in teacherBlockedBlocks]

    if lessonName in onlineLessons:
        candidateRooms = ['Online']  # aula online não ocupa sala física
    elif courseCode in classRooms and classRooms[courseCode]:
        candidateRooms = classRooms[courseCode]  # salas obrigatórias para esta UC
    else:
        candidateRooms = physicalRooms  # qualquer sala física disponível

    return [(block, room) for block in allowedBlocks for room in candidateRooms]


## Funções auxiliares para restrições

Estas funções são reutilizadas tanto em restrições fortes como suaves. Cada uma encapsula uma regra específica do horário.


In [ ]:
def noOverlappingBlock(*lessonAssignments):
    """Garante que todos os blocos são distintos (evita sobreposições)."""
    blocksUsed = [block for block, _ in lessonAssignments]
    return len(set(blocksUsed)) == len(blocksUsed)


def notSameRoom(leftAssignment, rightAssignment):
    """Impede que duas aulas físicas usem a mesma sala no mesmo bloco."""
    (blockLeft, roomLeft) = leftAssignment
    (blockRight, roomRight) = rightAssignment
    if roomLeft == 'Online' or roomRight == 'Online':
        return True
    return not (blockLeft == blockRight and roomLeft == roomRight)


def lessonsOnDiffDays(firstAssignment, secondAssignment):
    """Força as duas aulas de uma UC a ocorrerem em dias distintos."""
    return blockToDayMap[firstAssignment[0]] != blockToDayMap[secondAssignment[0]]


def maxFourDaysLessons(*lessonAssignments):
    """Exige que cada turma tenha aulas exatamente em quatro dias distintos."""
    daysWithClasses = {blockToDayMap[block] for block, _ in lessonAssignments}
    return len(daysWithClasses) == 4


def threeClassesMax(dayBlocks):
    """Cria uma restrição que limita a três o número de blocos ocupados num dia."""
    dayBlocksSet = set(dayBlocks)

    def withinLimit(*lessonAssignments):
        count = sum(1 for block, _ in lessonAssignments if block in dayBlocksSet)
        return count <= 3

    return withinLimit


def consecutiveClasses(*lessonAssignments):
    """Exige que aulas da mesma turma, no mesmo dia, ocupem blocos consecutivos."""
    dayToBlocks = {}
    for block, _ in lessonAssignments:
        day = blockToDayMap[block]
        dayToBlocks.setdefault(day, []).append(int(block[1:]))

    for blockNumbers in dayToBlocks.values():
        if len(blockNumbers) > 1:
            blockNumbers.sort()
            if max(blockNumbers) - min(blockNumbers) != len(blockNumbers) - 1:
                return False
    return True


### Restrição suave: reutilização de salas

Pretendemos minimizar o número de salas físicas que cada UC utiliza. Esta função conta quantas salas adicionais são necessárias além da primeira e servirá para calcular uma penalização suave.


In [ ]:
def extraRoomsUsed(*lessonAssignments):
    """Conta quantas salas físicas extra são usadas além da primeira."""
    physical_rooms = {room for _, room in lessonAssignments if room != 'Online'}
    if not physical_rooms:
        return 0
    return max(0, len(physical_rooms) - 1)


## Hard Constraints

A função seguinte adiciona ao problema todas as restrições obrigatórias: ausência de sobreposições por docente, por turma e por sala, além do limite de três blocos diários por turma.


In [ ]:
def addHardConstrains(problem):
    """Acrescenta restrições obrigatórias ao problema dado."""
    # Evita sobreposições para cada docente.
    for lecturerName in lecturers:
        variablesForLecturer = [
            f'{courseCode}_{index}'
            for courseCode in classLecturers[lecturerName]
            for index in (1, 2)
        ]
        problem.addConstraint(noOverlappingBlock, tuple(variablesForLecturer))

    # Evita sobreposições para cada turma.
    for groupName in classGroups:
        variablesForGroup = [
            f'{courseCode}_{index}'
            for courseCode in courseClasses[groupName]
            for index in (1, 2)
        ]
        problem.addConstraint(noOverlappingBlock, tuple(variablesForGroup))

    # Impede que duas aulas físicas partilhem a mesma sala/bloco.
    for leftLesson, rightLesson in combinations(lessonsToSchedule, 2):
        problem.addConstraint(notSameRoom, (leftLesson, rightLesson))

    # Limita cada turma a, no máximo, três blocos por dia.
    for groupName in classGroups:
        varsForGroup = [
            lesson
            for lesson in lessonsToSchedule
            if courseToGroup[lesson.split('_')[0]] == groupName
        ]
        for day, dayBlocks in daysBlocks.items():
            problem.addConstraint(threeClassesMax(dayBlocks), tuple(varsForGroup))


## Avaliação das soluções (Soft Constrains)

Esta função atribui penalizações quando deteta padrões indesejados. Isto serve como Soft Constrains uma vez que apesar de não serem restrições obrigatórias, queremos minimizá-las.


In [ ]:
def evaluateSolution(solution):
    """Calcula a penalização total associada a uma solução candidata."""
    penalty = 0

    # Penalizações ao nível da turma.
    for groupName in classGroups:
        groupLessons = [
            lesson for lesson in lessonsToSchedule
            if courseToGroup[lesson.split('_')[0]] == groupName
        ]
        assignments = [solution[lesson] for lesson in groupLessons]
        if not consecutiveClasses(*assignments):
            penalty += 10
        if not maxFourDaysLessons(*assignments):
            penalty += 100

    # Penalizações ao nível da UC.
    for courseCode in courses:
        assignments = [solution[f'{courseCode}_1'], solution[f'{courseCode}_2']]
        if not lessonsOnDiffDays(*assignments):
            penalty += 50
        penalty += 20 * extraRoomsUsed(*assignments)

    return penalty


## Procura da melhor solução

Usamos o `MinConflictsSolver` com várias inicializações para explorar o espaço de soluções. Mantemos a melhor solução encontrada com base na penalização calculada anteriormente. Uma vez que mesmo com o dataset mínimo, o domínio é muito grande, decidimos utilizar o `MinConflictsSolver`, que usa procura local, visto que qualquer solver com Backtracking levaria horas a encontrar uma única solução, tornando inviável a execução de múltiplas iterações para encontrar a melhor solução possível.


In [ ]:
bestSolution = None
"""Pusemos 10000 iterações mas isto vai claramente demorar muito tempo, mas vai obter um resultado melhor. Um número que demora menos tempo com um resultado OK seria ~3000 iterações"""
iterations = 10000
minScore = float('inf')

lessonDomains = {lesson: domainForLesson(lesson) for lesson in lessonsToSchedule}
orderedLessons = sorted(lessonsToSchedule, key=lambda lesson: len(lessonDomains[lesson]))

for iteration in range(iterations):
    solver = MinConflictsSolver()
    solver.random = random.Random(iteration)  # controla a aleatoriedade para repetibilidade

    timetableProblem = Problem(solver)
    for lesson in orderedLessons:
        timetableProblem.addVariable(lesson, lessonDomains[lesson])

    addHardConstrains(timetableProblem)
    solution = timetableProblem.getSolution()
    if not solution:
        continue  # nenhuma solução viável encontrada nesta iteração

    penalty = evaluateSolution(solution)
    if penalty < minScore:
        minScore = penalty
        bestSolution = solution
        print(f'Iteração {iteration + 1} de {iterations}, nova melhor penalização: {minScore}')
    elif (iteration + 1) % 200 == 0:
        if minScore == float('inf'):
            print(f'Iteração {iteration + 1} de {iterations}, ainda sem solução viável.')
        else:
            print(f'Iteração {iteration + 1} de {iterations}, melhor penalização até agora: {minScore}')

    if minScore == 0:
        break  # não há penalização; a solução cumpre todas as preferências, não vale a pena procurar mais


## Visualização do horário resultante

Formatamos a melhor solução encontrada para ajudar a validar o horário: primeiro organizado por dia/bloco, depois numa lista ordenada pelos blocos.


In [ ]:
print(f"\nMelhor solução com penalização {minScore}:\n")

if not bestSolution:
    print('Sem solução encontrada.')
else:
    schedule = {day: {block: [] for block in daysBlocks[day]} for day in days}
    for lesson, (block, room) in bestSolution.items():
        course = lesson.split('_')[0]
        entry = {
            'lesson': lesson,
            'course': course,
            'group': courseToGroup[course],
            'teacher': courseToTeacher[course],
            'room': room,
            'block': block,
            'day': blockToDayMap[block],
        }
        schedule[entry['day']][block].append(entry)

    for day in days:
        print(f"\n=== {day} ===")
        for block in daysBlocks[day]:
            entries = schedule[day][block]
            if not entries:
                continue
            print(f"{block}:")
            for e in sorted(entries, key=lambda x: (x['room'], x['course'])):
                print(f"  {e['room']}: {e['course']} ({e['group']}, {e['teacher']})")

    print("\n--- Lista por bloco ---")
    ordered = sorted(
        [
            {
                'lesson': lesson,
                'course': lesson.split('_')[0],
                'group': courseToGroup[lesson.split('_')[0]],
                'teacher': courseToTeacher[lesson.split('_')[0]],
                'room': room,
                'block': block,
                'day': blockToDayMap[block],
            }
            for lesson, (block, room) in bestSolution.items()
        ],
        key=lambda e: int(e['block'][1:]),
    )
    for e in ordered:
        print(f"{e['block']} ({e['day']}): {e['room']} - {e['course']} ({e['group']}, {e['teacher']})")



## Conclusão

O processo de desenvolvimento demonstrou que o modelo de CSP implementado neste notebook responde de forma eficaz às restrições impostas pelas turmas, docentes e salas. O solver MinConflictsSolver, disponibilizado pela biblioteca python-constraint, revelou-se adequado para obter horários consistentes e com penalizações reduzidas.

A estrutura modular do caderno, organizada em secções temáticas, facilitou a experimentação incremental e pode ser facilmente adaptada a outros cenários com pequenas alterações nos dados de entrada. Para um volume de cerca de 5000 iterações, a execução demora aproximadamente três minutos, valor considerado aceitável face à natureza iterativa do algoritmo e à dimensão do domínio analisado.

Ainda assim, existem margens claras para evolução. Bibliotecas como o OR-Tools, poderiam explorar o espaço de soluções com maior eficiência. Estratégias adicionais de otimização do domínio poderiam também melhorar a performance do algoritmo.
